# This script process the baseline results 

1. Compile the baseline with:
```
make bin/baseline
make bin/par_baseline
```
2. Run the benchmarking script
```
cd scripts
python3 run_par_baseline
```
3. The output that we will process is baseline_coffee.txt (renamed from baseline.out)

In [ ]:
# select the server_name to use it as a suffix for the file name
server_name = "alder"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean

experiment ={}

with open(f"baseline_{server_name}.txt") as f:
    for line in f:
        tokens = line.split()
        if "Running:" in tokens:
            name=tokens[2].split("/")[3]
            nth=int(tokens[6])
            if name not in experiment:
                experiment[name]={'seq':{}, 'par':{} }
        if "SEQUENTIAL" in tokens:
            status="seq"
        if 'CORES' in tokens:
            status="par"
        if 'Octree' in tokens:
            experiment[name][status][nth]=[float(tokens[5])]
        if "STAGE" in tokens:
            experiment[name][status][nth].append(float(tokens[5]))
        if 'Average:' in tokens:
            experiment[name][status][nth].append(float(tokens[1]))

#print(experiment)

results = {}

for i in experiment:
    results[i]={'seq':{}, 'par':{} }
    results[i]['seq']['octree']=experiment[i]['seq'][1][0]
    results[i]['seq']['stage1']=mean(experiment[i]['seq'][1][1:15:3])
    results[i]['seq']['stage2']=mean(experiment[i]['seq'][1][2:15:3])
    results[i]['seq']['stage3']=mean(experiment[i]['seq'][1][3:15:3])
    results[i]['seq']['owm']=experiment[i]['seq'][1][16]
    for j in experiment[i]['par']:
        results[i]['par'][j]={}
        results[i]['par'][j]['octree']=experiment[i]['par'][j][0]
        results[i]['par'][j]['stage1']=mean(experiment[i]['par'][j][1:15:3])
        results[i]['par'][j]['stage2']=mean(experiment[i]['par'][j][2:15:3])
        results[i]['par'][j]['stage3']=mean(experiment[i]['par'][j][3:15:3])
        results[i]['par'][j]['owm']=experiment[i]['par'][j][16]
#print(results)
for i in experiment:
    print("Cloud {} has Octree creation time = {:.2f}".format(i,mean(list(results[i]['par'][j]['octree'] for j in [1,2,4,6,8]))))


In [ ]:
for i in experiment:
    print("Cloud {} has sequential OWM time = {:.2f}".format(i,results[i]['seq']['owm'] ))


In [ ]:
for i in experiment:
    print("Cloud {} has parallel OWM with 8 threads time(speedup) = {:.2f} ({:.1f}x)".format(i,results[i]['par'][8]['owm'],results[i]['seq']['owm']/results[i]['par'][8]['owm'] ))

In [ ]:
for i in experiment:
    for j in ['stage1','stage2','stage3']:
        print("Cloud {} stage {} time (percent) = {:.4f} ({:.2f}%)".format(i,j,results[i]['seq'][j],results[i]['seq'][j]/results[i]['seq']['owm']*100))


# Spedup for each cloud

In [ ]:
#Configuration variables
titlefs = 20
ylabelfs = 18
xlabelfs = 18
xticksfs = 16
yticksfs = 16
legendfs = 14
linew = 2
markers = 8

fig = plt.figure()

marks=['o-','x-','s-','v-','+-']

for (i,z) in zip(experiment,marks):
    print(np.array([results[i]['seq']['owm']/results[i]['par'][j]['owm'] for j in [1,2,4,6,8]]))
    plt.plot(np.array([1,2,4,6,8]), np.array([results[i]['seq']['owm']/results[i]['par'][j]['owm'] for j in [1,2,4,6,8]]), z, linewidth=linew, markersize=markers)
plt.plot(np.array([1,2,4,6,8]), np.array([1,2,4,6,8]), '-', linewidth=linew, markersize=markers)

sizes=[i for i in experiment]
sizes.append('Ideal')
plt.title('Speedup',  fontweight='bold', fontsize=titlefs)
plt.legend(sizes,loc='best', fontsize= legendfs)
plt.ylabel('Speedup', fontsize=ylabelfs)
plt.xlabel('Number of cores', fontsize=xlabelfs)
plt.xticks([1,2,4,6,8],fontsize=xticksfs)
plt.yticks(fontsize=yticksfs)
plt.grid()

plt.show()

In [ ]:
pp = PdfPages(f"Speedup-ompBaseline-{server_name}.pdf")
pp.savefig(fig)
pp.close()

# Start incremental file with results for each optimization

In [ ]:
output=f"All_Optimizations-{server_name}.csv"

f = open(output, "a")
f.write("Optimization;Cloud;TimeTree;TimeOWM;Level;MinRadMaxNum\n")

for i in experiment:
    f.write("Baseline;{};{:.5f};{:.5f};{};{}\n".format(i,mean(list(results[i]['par'][j]['octree'] for j in [1,2,4,6,8])),results[i]['par'][8]['owm'],0,0))

f.close()